In [33]:
from src.ml.model import AlphaPolicyModel, AlphaValueModel
from src.connect4.agent import RandomAgent, MCTSAgent
from src.connect4.game import run
import time
from src.ml.train_util import record_tree
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from src.ml.model import AlphaPolicyModel, AlphaValueModel
from src.connect4.agent import RandomAgent, MCTSAgent
from src.connect4.game import run
import time
from src.ml.train_util import record_tree

In [ ]:
alpha_value = AlphaValueModel()
alpha_policy = AlphaPolicyModel()

agent_1 = MCTSAgent(
    value_network=alpha_value,
    policy_network=alpha_policy
)

agent_2 = RandomAgent()

start = time.time()
run(agent_1, agent_2)
print(f' {time.time() - start} s')